# Raziskovalna analiza podatkov
## HiggsML Challenge
Klemen Kenda, april 2016

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importing data

In [2]:
dirName = '../../data/'
fileName = dirName + 'training.csv'
data = pd.read_csv(fileName)

# replace missing values with NaN
data = data.replace(-999.0, np.nan)

#### Initial data analysis

In [3]:
# number of rows
rows = data.shape[0]

# make separate variables for signal and background
signal = data.query('Label == "s"')
background = data.query('Label == "b"')

# number of signal rows
signalRows = signal.shape[0]
# number of background rows
backgroundRows = background.shape[0]

In [4]:
{rows, signalRows, backgroundRows}

{85667, 164333, 250000}

In [5]:
# by label
byLabel = data.groupby('Label')

# write to file
# TODO 
#   - smaller fonts
#   - different arrangement
fo = open("byLabelDescriptionStats.tex", "w")
fo.write(byLabel.describe().transpose().to_latex())
fo.close()

fo = open("dataDescriptionStats.html", "w")
fo.write(data.describe().transpose().to_html())
fo.close()

## Histograms

In [6]:
def DoubleHist(var, rMin, rMax):
    plt.figure(var);
    num_bins = 30;
    x = background[var]
    y = signal[var]
    common_params = dict(bins = 50, range=(rMin, rMax), normed=1)
    n, bins, patches = plt.hist(x, facecolor='green', alpha=0.5, **common_params)
    n, bins, patches = plt.hist(y, facecolor='orange', alpha=0.5, **common_params)
    plt.title(var)
    plt.savefig('ea_hist_' + var.lower() + '.pdf')

In [ ]:
# histograms
DoubleHist('Weight', 0, 8)
DoubleHist('DER_mass_MMC', 0, 400)
DoubleHist('DER_mass_transverse_met_lep', 0, 200)
DoubleHist('DER_mass_vis', 0, 300)
DoubleHist('DER_pt_h', 0, 400)
DoubleHist('DER_deltaeta_jet_jet', 0, 9)
DoubleHist('DER_mass_jet_jet', 0, 2500)
DoubleHist('DER_prodeta_jet_jet', -20, 15)
DoubleHist('DER_deltar_tau_lep', 0, 6)
DoubleHist('DER_pt_tot', 0, 100)
DoubleHist('DER_sum_pt', 0, 1000)
DoubleHist('DER_pt_ratio_lep_tau', 0, 7)
DoubleHist('DER_met_phi_centrality', -1.5, 1.5)
DoubleHist('DER_lep_eta_centrality', 0, 1)

DoubleHist('PRI_tau_pt', 0, 180)
DoubleHist('PRI_tau_eta', -2.5, 2.5)
DoubleHist('PRI_tau_phi', -3.2, 3.2)

DoubleHist('PRI_lep_pt', 0, 200)
DoubleHist('PRI_lep_eta', -2.5, 2.5)
DoubleHist('PRI_lep_phi', -3.2, 3.2)

DoubleHist('PRI_met', 0, 300)
DoubleHist('PRI_met_phi', -3.2, 3.2)
DoubleHist('PRI_met_sumet', 0, 1000)

DoubleHist('PRI_jet_num', 0, 3)

DoubleHist('PRI_jet_leading_pt', 0, 500)
DoubleHist('PRI_jet_leading_eta', -4.5, 4.5)
DoubleHist('PRI_jet_leading_phi', -3.2, 3.2)

DoubleHist('PRI_jet_subleading_pt', 0, 300)
DoubleHist('PRI_jet_subleading_eta', -4.5, 4.5)
DoubleHist('PRI_jet_subleading_phi', -3.2, 3.2)

DoubleHist('PRI_jet_all_pt', 0, 400)

## Correlation Matrix

In [81]:
import seaborn as sns
from sklearn.cluster import KMeans

In [43]:
corrData = data.copy()
del corrData['EventId']
del corrData['Weight']
cm = corrData.corr()
f, ax =  plt.subplots(figsize=(12, 9))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
plt.setp(ax.yaxis.get_majorticklabels(), rotation=90)
sns.heatmap(cm, vmax=.8,square=1)
f.savefig('ea_corr_data.pdf')

In [44]:
cg = sns.clustermap(cm, vmax = .8, square=1)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)
cg.savefig('ea_corr_clustermap.pdf')

## Scatter Plots

In [78]:
# for correlated
df = data[['DER_mass_jet_jet', 'DER_lep_eta_centrality', 'DER_deltaeta_jet_jet', 'DER_prodeta_jet_jet', 'Label']].sample(n = 2000)

In [86]:
color_wheel = {"s": "#ff0000", "b": "#99ff99"}
colors = df["Label"].map(lambda x: color_wheel.get(x))
ax = scatter_matrix(df, color=colors, alpha=0.5, figsize=(15, 15), diagonal='kde')

In [85]:
plt.show()

In [88]:
# for correlated - top
df = data[['DER_sum_pt', 'PRI_jet_all_pt', 'PRI_met_sumet', 'DER_pt_h', 'Label']].sample(n = 2000)

In [89]:
color_wheel = {"s": "#ff0000", "b": "#99ff99"}
colors = df["Label"].map(lambda x: color_wheel.get(x))
ax = scatter_matrix(df, color=colors, alpha=0.5, figsize=(15, 15), diagonal='kde')

In [90]:
plt.show()

In [91]:
# for non correlated
df = data[['DER_sum_pt', 'DER_lep_eta_centrality', 'DER_pt_ratio_lep_tau', 'PRI_lep_eta', 'Label']].sample(n = 2000)

In [92]:
color_wheel = {"s": "#ff0000", "b": "#99ff99"}
colors = df["Label"].map(lambda x: color_wheel.get(x))
ax = scatter_matrix(df, color=colors, alpha=0.5, figsize=(15, 15), diagonal='kde')

In [93]:
plt.show()

## No Value

In [126]:
# generate table with missing value data
numS = signal.shape[0]
signalC = signal.isnull().sum()/numS
numB = background.shape[0]
backgroundC = background.isnull().sum()/numB
result = pd.concat([signalC, backgroundC], axis=1)
resultF = result[(result[1] != 0)]
fo = open('exploratoryNaNValues.tex', 'w')
fo.write(resultF.to_latex())
fo.close()

In [127]:
# mean for missing values per type
resultF.mean()

0    0.479367
1    0.624191
dtype: float64